imports

In [ ]:
import pandas as pd

config

In [ ]:
config_prefix = ['BR', 'BI', 'TR', 'TI']

config_datasheet = r'./preprocessed.xlsx'

import df

In [3]:
raw = pd.read_excel(config_datasheet)
df = pd.DataFrame(raw)
df = df.loc[df["Flag"]!="outlier"]

get names of algorithm

In [4]:
algo_name_array = [name for name in df["Algorithm"].unique()]

split dataframes 

In [5]:
df_cat_array = []
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="BU")])
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="BU")])
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="TD")])
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="TD")])

df_algo_cat_array = []
for algo_name in algo_name_array:
    tmp_df = []
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="BU")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="BU")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="TD")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="TD")])
    df_algo_cat_array.append(tmp_df)

create result

In [6]:
cols = ["Algorithm", "Comprehension",
        "R_Response_count", "R_Correctness", "R_ResponseTimeMean", "R_ResponseTimeDeviation",
        "I_Response_count", "I_Correctness", "I_ResponseTimeMean", "I_ResponseTimeDeviation"]

result_df = pd.DataFrame(columns=cols)

for algo_df_array in df_algo_cat_array:
    for i in range(2):
        rec =  algo_df_array[i*2+0]
        it =  algo_df_array[i*2+1]
        
        Algorithm = rec.iloc[0]["Algorithm"]
        Comprehension = "Top-Down"
        if i == 1:
            Comprehension = "Bottom-Up"
        
        R_Response_count = len(rec)
        R_Correctness = float(len(rec.loc[rec["Correctness"]==True]))/float(len(rec))
        R_ResponseTimeMean = int(rec["ResponseTime"].mean())
        R_ResponseTimeDeviation = None
        
        I_Response_count = len(it)
        I_Correctness = float(len(it.loc[it["Correctness"]==True]))/float(len(it))
        I_ResponseTimeMean = int(it["ResponseTime"].mean())
        I_ResponseTimeDeviation = None
        
        data = [
            Algorithm, Comprehension,
            R_Response_count, R_Correctness, R_ResponseTimeMean, R_ResponseTimeDeviation,
            I_Response_count, I_Correctness, I_ResponseTimeMean, I_ResponseTimeDeviation,
        ]
        
        result_df = result_df.append(pd.DataFrame([data], columns=cols))

for i in range(2):
    rec =  df_cat_array[i*2+0]
    it =  df_cat_array[i*2+1]
    
    Algorithm = "Total"
    Comprehension = "Top-Down"
    if i == 1:
        Comprehension = "Bottom-Up"
    
    R_Response_count = len(rec)
    R_Correctness = float(len(rec.loc[rec["Correctness"]==True]))/float(len(rec))
    R_ResponseTimeMean = int(rec["ResponseTime"].mean())
    R_ResponseTimeDeviation = int(rec["ResponseTime"].std())
    
    I_Response_count = len(it)
    I_Correctness = float(len(it.loc[it["Correctness"]==True]))/float(len(it))
    I_ResponseTimeMean = int(it["ResponseTime"].mean())
    I_ResponseTimeDeviation = int(it["ResponseTime"].std())
    
    data = [
        Algorithm, Comprehension,
        R_Response_count, R_Correctness, R_ResponseTimeMean, R_ResponseTimeDeviation,
        I_Response_count, I_Correctness, I_ResponseTimeMean, I_ResponseTimeDeviation,
    ]
    
    result_df = result_df.append(pd.DataFrame([data], columns=cols))
    
result_df.to_excel("response_times.xlsx")